# Scraping ISFC data

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, parse_qs
import pandas as pd
import time
import os
import re

In [22]:
def link_generator(original_link):
    try:
        # Parse the URL
        parsed_url = urlparse(original_link)

        # Extract parameters
        query_params = parse_qs(parsed_url.query)
        event_number = query_params.get('event', [None])[0]
        result_number = query_params.get('result', [None])[0]

        # Check if the required parameters are present
        if event_number is not None and result_number is not None:
            # Convert to integers if needed
            event_number = int(event_number)
            result_number = int(result_number)

            # Create the second URL
            return f'https://components.ifsc-climbing.org/result-complete/?event={event_number}&result={result_number}'
        else:
            return 'Error: Missing required parameters (event or result) in the URL.'

    except Exception as e:
        return f'Error: {e}'

In [32]:
def get_competition_data(year):
    menu = Select(driver.find_element(By.ID, "yearSelect"))
    menu.select_by_visible_text(year)
    driver.implicitly_wait(10)

    print("YEAR", year)
    
    # Store information of each competition
    world_cups_list = []

    # Find competitions by class name
    competitions = driver.find_elements(By.CLASS_NAME, 'competition')

    for c in competitions:
        # print(c.find_element(By.CLASS_NAME, 'title').text)
        # print(c.find_element(By.CLASS_NAME, 'date').text)
        title = c.find_element(By.CLASS_NAME, 'title').text
        date = c.find_element(By.CLASS_NAME, 'date').text
        
        # Get categories (BOULDER, LEAD, SPEED)
        categories = c.find_elements(By.CLASS_NAME, 'tag')

        competition_entry = {}

        # TODO: MAKE ERROR HANDLER WHEN THERE IS TAG BUT THERE IS NO HREF
        if categories:
            for cat in categories:
                category_name = cat.text

                try:
                    # Attempt to find the 'a' tag
                    a_tag = cat.find_element(By.TAG_NAME, 'a')
                    href_link = a_tag.get_attribute("href")
                    new_href = link_generator(href_link)
                except NoSuchElementException:
                    # Handle the case when 'a' tag is not found
                    break

                # Store competition information as dictionary
                competition_entry = {
                    'title': title,
                    'date': date,
                    'category': category_name,
                    'url': new_href
                }

                # Append entry to world cup list
                world_cups_list.append(competition_entry)
    
    return world_cups_list

In [24]:
def save_csv(comp_list, year):
    # Create a DataFrame from the competition data
    df = pd.DataFrame(comp_list)
    csv_file_path = f'world_cups_{year}.csv'

    df.to_csv(csv_file_path, index=False, encoding='utf-8')

### First Step: Get competition information

In this step, our job is to scrape the calendar page to get all the worldcups information available such as `event name`, `date`, the `categories` played on that event year, and the `url` for each category competition.

Each competition information will be save on its own csv file by year. 

In [33]:
options = Options()
options.add_argument('--headless')
# options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)


driver.get("https://components.ifsc-climbing.org/calendar/")

In [34]:
# Find dropdown menu using its ID
dropdown = driver.find_element(By.ID, "yearSelect")

# Get all years in the dropdown menu using CSS selector
values = dropdown.find_elements(By.CSS_SELECTOR, 'option')
years = []

for v in values:
    # There is not information for worldcups on the ifsc calendar website after 2007
    if v.text == '2006':
        break
    
    years.append(v.text)

# Loop through years
for year in years:
    try:
        # Find the dropdown menu each time before selecting a year
        dropdown = driver.find_element(By.ID, "yearSelect")
        
        # Select the year from the dropdown
        menu = Select(dropdown)
        menu.select_by_visible_text(year)

        # Wait for some time to let the page load
        time.sleep(2)

        # Get competition data for the selected year
        competition_list = get_competition_data(year)
        save_csv(competition_list, year)

    except StaleElementReferenceException:
        # Handle StaleElementReferenceException by refreshing the dropdown
        print("StaleElementReferenceException: Refreshing dropdown.")
        continue

driver.quit()

YEAR 2024
YEAR 2023
YEAR 2022
YEAR 2021
YEAR 2020
YEAR 2019
YEAR 2018
YEAR 2017
YEAR 2016
YEAR 2015
YEAR 2014
YEAR 2013
YEAR 2012
YEAR 2011
YEAR 2010
YEAR 2009
YEAR 2008
YEAR 2007


### Second Step: Scrape Competition Results for each Year

The second step involves scrape the competition results from each url obtained on the first step.

The `extract_event_title` function accepts a string, which in this case will be the event name from each csv file (like IFSC Climbing Worldcup (B) - Erlangen (GER) 2007), and extract the city, country, and year, then returns the new created string, in this case Erlangen_(GER)_2007, to be used as  file name to store competition results for that specific event.

In [2]:
def extract_event_title(input_string):
    # Define the pattern to match location and year between hyphens
    pattern = r'[-]\s*([^-.]+(?:\([^)]+\))?)\s*[-]?(\d{4})'

    # Use re.search to find the pattern in the input string
    match = re.search(pattern, input_string)

    # Check if a match is found
    if match:
        location = match.group(1).strip()
        location_with_underscore = location.replace(' ', '_')
        country_and_year = f"{location_with_underscore}_{match.group(2)}"
        return country_and_year
    else:
        return input_string.replace(' ', '_')

The `scrape_tables` function accepts a string, which is the url string for a competition, and extracts the table results for that event, and returns a pandas data frame with the tables resutls in it.

In [3]:
def scrape_tables(url):
    options = Options()
    options.add_argument('--headless')
    # options.add_argument('--incognito')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    driver.implicitly_wait(10)

    try:
        # Find the table element
        table = driver.find_element(By.CSS_SELECTOR, 'table')

        # Initialize empty lists to store table data
        table_header = []
        table_data = []

        # Find the header row (within the thead section)
        thead = table.find_element(By.CSS_SELECTOR, 'thead')
        header_row = thead.find_element(By.CSS_SELECTOR, 'tr')

        # Extract header cell text
        for cell in header_row.find_elements(By.CSS_SELECTOR, 'th'):
            header_text = cell.text
            table_header.append(header_text)

        # Find and iterate through data rows (within the tbody section)
        tbody = table.find_element(By.CSS_SELECTOR, 'tbody')
        for row in tbody.find_elements(By.CSS_SELECTOR, 'tr'):
            row_data = []

            # Iterate through columns in the current row
            for cell in row.find_elements(By.CSS_SELECTOR, 'td'):
                # Extract text from each cell
                cell_text = cell.text
                row_data.append(cell_text)

            # Add the row data to the table data list
            table_data.append(row_data)
        
        # Close the browser  
        driver.quit()

        # Check if there is any data in table
        if not table_data:
            print(f"No table data on page.{url}")
            return None
        
        # Create a Pandas DataFrame from the table data
        df = pd.DataFrame(table_data, columns=table_header)

        return df
    
    except NoSuchElementException:
        print("No table element found on the page.")
        driver.quit()
        return None

To get the tables results from each url competition, I will have to:
- Get a list of the world cups csv files (from step one)
- For each competition year, make a folder to store all world cups played on that year with its different categories in a csv file.
- The folder structure should look like this:
    * world_cups_`{year}`/
        - lead_`{year}`.csv
        - boulder_`{year}`.csv
        - speed_`{year}`.csv
        - combined_`{year}`.csv
        - Boulder&Lead_`{year}`.csv
- Since there are a lot of competitions spanning from 2007 to 2023, create a file to keep track of the url scraped in case something fails or we want to stop and then later keep scraping the data.

In [10]:
folder_path = "./world_cups/"
progress_file_path = "./progress.txt"

# Check if progress file exists
if os.path.exists(progress_file_path):
    with open(progress_file_path, 'r') as progress_file:
        last_processed_file = progress_file.readline().strip()
        last_processed_index = int(progress_file.readline().strip())
else:
    last_processed_file = None
    last_processed_index = 0

# Get all csv files name from folder
csv_files_list = os.listdir(folder_path)
csv_files_list.sort()

# Find the index of the last processed file
if last_processed_file:
    try:
        start_index = csv_files_list.index(last_processed_file)
    except ValueError:
        start_index = 0
else:
    start_index = 0

# Iterate through each csv file
for file_name in csv_files_list[start_index:]:
    df = pd.read_csv(os.path.join(folder_path, file_name))

    # Extract the year from file name
    year = file_name.split('_')[-1]
    year = year.split('.')[0]
    
    # Get the event name, category and url from csv file
    for i in range(last_processed_index, len(df)):
        event_name = extract_event_title(df['title'][i])
        event_category = df['category'][i]
        event_date = df['date'][i]
        url = df['url'][i]
        gender = df['category'][i].split(' ')[1]

        # Get table result from each url
        scraped_df = scrape_tables(url)

        if scraped_df is None:
            break

        scraped_df['event_name'] = event_name
        scraped_df['date'] = event_date
        scraped_df['gender'] = gender

        folder_name = file_name.rsplit('.', 1)[0]

        # Create folder
        if not os.path.exists(f'./{folder_name}'):
            os.makedirs(f'./{folder_name}')

        # Extract result number from the URL
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        result_number = query_params.get('result', [None])[0]

        # Check if the required parameters are present
        if result_number is not None:
            # Convert to integers if needed
            result_number = int(result_number)

        # Determine the category
        if result_number == 1 or result_number == 5:
            category = 'lead'
        elif result_number == 2 or result_number == 6:
            category = 'speed'
        elif result_number == 3 or result_number == 7:
            category = 'boulder'
        elif result_number == 4 or result_number == 8:
            category = 'combined'
        elif result_number == 617 or result_number == 618:
            category = 'boulder_lead'

        # Create or open CSV file for the specific category
        file_path = os.path.join(f'./{folder_name}', f'{year}_{category}.csv')
        
        if os.path.exists(file_path):
            # Append to existing file
            scraped_df.to_csv(file_path, mode='a', header=False, index=False, encoding='utf-8')
        else:
            # Create new file
            scraped_df.to_csv(file_path, index=False, encoding='utf-8')
            
        print(f"Saving {file_path}")

        # Update progress file after processing each row
        with open(progress_file_path, 'w') as progress_file:
            progress_file.write(f"{file_name}\n")
            progress_file.write(f"{i+1}\n")

# Remove progress file after completion
os.remove(progress_file_path)

Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_boulder.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_lead.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_2007/2007_speed.csv
Saving ./world_cups_20